# Apple stock price prediction

We are goin to predict the stock price of Apple using historic data.
I have collected the stock price history data using 

from Yahoo Finance

I have used SMA(Simple moving average) and EMA(Exponential moving average) as a parameter for prediction.


I have found the SMA5,SMA10,SMA50,SMA100,EMA5,EMA10,EMA50,EMA100 for the data which is shown below

In [1]:
import pandas as pd
data = pd.read_csv('AAPL.csv')
data.describe()

,SMA5,EMA5,SMA10,EMA10,SMA50,EMA50,SMA100,EMA100,Close
count,9365.000000,9365.000000,9365.000000,9365.000000,9365.000000,9365.000000,9365.000000,9365.000000,9365.000000
mean,24.204763,24.204813,24.155632,24.155246,23.755437,23.762918,23.283560,23.292930,24.244365
std,42.651165,42.648662,42.566658,42.559835,41.857538,41.858703,41.060230,41.032507,42.720298
min,0.203571,0.206450,0.212946,0.213186,0.231786,0.238114,0.252277,0.055566,0.196429
25%,1.060714,1.061717,1.059714,1.061652,1.079600,1.088839,1.087386,1.124608,1.062500
50%,1.660714,1.659440,1.654018,1.657850,1.634229,1.645263,1.623393,1.635647,1.660714
75%,24.287429,24.291033,24.243857,24.300811,23.692514,23.570622,22.480071,22.521182,24.280001
max,192.856000,192.396267,191.953000,191.242939,184.273400,184.459328,178.387800,180.050818,193.979996


In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pandas as pd

### Importing data and spliting into test and training data

In [6]:
train_data =  data.head(7492)
test_data = data.tail(1873)

Now we are scaling the data to a certain range so that training will be perfect

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

Setting the hyper parameters and defining number of nodes

In [9]:
learning_rate = 0.03

timesteps = 1
num_input = 8

hidden_layer = 250

num_steps = 1500
batch_size = 7  

num_output = 1

Since Tensorflow is not imperative we have to declare variables and placeholders ahead

In [10]:
X = tf.placeholder(tf.float32 , [None,timesteps,num_input])
y = tf.placeholder(tf.float32, [None,num_output])
weight = {
	'weigh' : tf.Variable(tf.random_normal([hidden_layer, num_output]), name = 'weight')
}

bias = {
	'bie' : tf.Variable(tf.random_normal([num_output]), name = 'bias')
}


We are predicting the future stock price using Recurrent Neural Network for that we have to provide the input in batches so

In [12]:
def next_batch(data, timesteps):
	rand_start = np.random.randint(0,len(train_data)-timesteps)
	y_batch = np.array(train_data[rand_start:rand_start+timesteps])
	return y_batch[:,:-1].reshape(1,1,8),y_batch[:,-1].reshape(1,1)


We are defining the RNN

In [21]:
cell = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicLSTMCell(num_units = hidden_layer, activation = tf.nn.relu), output_size = num_output)
outputs, states = tf.nn.dynamic_rnn(cell, X,dtype = tf.float32)
loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train = optimizer.minimize(loss)

ValueError: Variable rnn/output_projection_wrapper/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/tensorflow/contrib/rnn/python/ops/core_rnn_cell.py", line 382, in call
    output, res_state = self._cell(inputs, state)
  File "<ipython-input-13-4f9de0fe6f77>", line 3, in <module>
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype = tf.float32)
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2878, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
